# Day 16: Ticket Translation
---
Hari ini terlalu banyak menggunakan *list comprehension*.

In [1]:
my_input = None
with open("input.txt") as file:
    my_input = file.read()


In [2]:
def parseInput(input_str):
    field_rules, your_ticket, nerby_tickets = input_str.strip().split("\n\n")
    field_rules = [parseFieldRule(field_rule) for field_rule in field_rules.split("\n")]
    your_ticket = tuple(int(n) for n in your_ticket.split("\n")[1].split(","))
    nerby_tickets = [tuple(int(n) for n in ticket.split(",")) for ticket in nerby_tickets.split("\n")[1:]]
    return field_rules, your_ticket, nerby_tickets

def parseFieldRule(rule_str):
    field, ranges = rule_str.split(": ")
    ranges = [tuple(int(n) for n in r.split("-")) for r in ranges.split(" or ")]
    rules = lambda x: any(_min_ <= x <= _max_ for _min_, _max_ in ranges)
    return field, rules

---
## Part 1
Bagian pertama cukup sederhana, cukup lakukan loop semua *value* dan lakukan pengecekan dari *field rules*.

In [3]:
def part1(input_str):
    field_rules, _, nearby_tickets = parseInput(input_str)
    check_valid_field = lambda value: any(rule(value) for field, rule in field_rules)
    invalid_values = [value for ticket in nearby_tickets for value in ticket if not check_valid_field(value)]
    return sum(invalid_values)

# Unit test

test_input1 = """
class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12
"""

# In this example, the values on the first nearby ticket are all valid for at least one field. 
# This is not true of the other three nearby tickets: the values 4, 55, and 12 are are not valid for any field.
# Adding together all of the invalid values produces your ticket scanning error rate: 4 + 55 + 12 = 71.

part1(test_input1) == 71

True

In [4]:
%time part1(my_input)

Wall time: 7.99 ms


21081

---
# Part 2
Bagian kedua kita saring tiket yang valid, kemudian masing-masing *field* cari kemungkinan index yang memenuhi aturannya. Dari kemungkinan index tersebut cari yang paling sesuai, dimulai dengan kemungkinan index yang terkecil.

In [5]:
import math

def part2(input_str, word):
    field_rules, your_ticket, nearby_tickets = parseInput(input_str)

    # Filter valid tickets
    check_valid_ticket = lambda t: all(any(r(v) for _, r in field_rules) for v in t)
    valid_tickets = [ticket for ticket in nearby_tickets if check_valid_ticket(ticket)]

    # Find all possible valid index for each field
    field_indexes = dict()
    for field, rule in field_rules:
        field_indexes[field] = []
        for i in range(len(field_rules)):
            if all(rule(ticket[i]) for ticket in valid_tickets):
                field_indexes[field] += [i]
    
    # Find the most possible index by sort the possible indexes
    assigned_index = []
    field_index = dict()
    for field, indexes in sorted(field_indexes.items(), key=lambda item: len(item[1])):
        indexes = [index for index in indexes if index not in assigned_index]
        assigned_index += indexes
        field_index[field] = indexes[0]
    
    # Return the product of field's value of specified word.
    return math.prod(your_ticket[index] for field, index in field_index.items() if field.startswith(word))

# Unit test

test_input2 = """
class: 0-1 or 4-19
row: 0-5 or 8-19
seat: 0-13 or 16-19

your ticket:
11,12,13

nearby tickets:
3,9,18
15,1,5
5,14,9
"""

# Based on the nearby tickets in the above example, the first position must be row, the second position must be class,
# and the third position must be seat; you can conclude that in your ticket, class is 12, row is 11, and seat is 13.

part2(test_input2, "class") == 12 and part2(test_input2, "row") == 11 and part2(test_input2, "seat") == 13

True

In [6]:
%time part2(my_input, "departure")

Wall time: 54 ms


314360510573